# 1 Import and init

In [1]:
#git push origin dev_models

In [2]:
import warnings

# Ignorer tous les avertissements
warnings.filterwarnings("ignore")

In [6]:
import sys
sys.path.append('/home/onyxia/work/cine-insights/src/models')
sys.path.append('/home/onyxia/work/cine-insights/src/data')
sys.path.append('/home/onyxia/work/cine-insights/src/features')
sys.path.append('/home/onyxia/work/cine-insights/models')


In [4]:
import pandas as pd
import joblib
import numpy as np
python_interpretor="/home/onyxia/work/TMDb-Box-Office-Prediction/Codes/BOF_ENv/bin/python"

In [35]:
 #
import Preprocess_train_data         
from Preprocess_train_data import * 
#
import  TRain  
from TRain import *
#
import Preprocess_predicted_data  # requirements train_model_xgboot22a6 and  Build_Feats1a
from  Preprocess_predicted_data import *  #  Utilisez les fonctions du module file

import Predict__Model  # requirements train_model_xgboot22a6 and  Build_Feats1a
from  Predict__Model import *  #  Utilisez les fonctions du module file

In [11]:
filename="Raw_df_50_ligne.csv"
path_dir="/home/onyxia/work/cine-insights/src/data/"

#  2 Load Data

In [12]:
raw_df=pd.read_csv(path_dir+filename)

In [13]:
raw_df.head(1)

,Unnamed: 0,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,Keywords
0,0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."


In [ ]:
# 3 Preprocessing 

# 3 Build model 

## 3.1 initialize an instance of model class

In [14]:
model=TRain.XGBRegressorWrapper(raw_df)


(x.shape, y.shape ) 

 Pour le train : (37, 24) (37,) 

 Pour le test : (11, 24) (11,) 

 Pour la validation : (2, 24) (2,) 

Fitting 2 folds for each of 162 candidates, totalling 324 fits


In [53]:
# 3.2 Entrainement model
#model._train_model()

## 3.2  Get pipeline Transformator

In [15]:
ct_data_transformer=model.ct_data_transformer

## 3.3 Get performances

In [16]:
model.display_results()

,Best parameters,Training RMSLE,Test RMSLE,Test MAPE,Training time
0,"{'colsample_bytree': 0.7, 'learning_rate': 0.0...",-0.135773,4 NaN 12 ...,63.959912,4.9643


## 3.4 Predictions

## 3.4.1 Predictions on already new transformed data

In [17]:
X_test=model.X_test

In [18]:
X_test.columns # Xval a été bien preprocessé donc on peut l'utiliser directement comme argument de la fonction predict

Index(['budget', 'popularity', 'has_homepage', 'release_month', 'release_year',
       'genre_Action', 'genre_Adventure', 'genre_Animation', 'genre_Comedy',
       'genre_Crime', 'genre_Drama', 'genre_Family', 'genre_Fantasy',
       'genre_Mystery', 'genre_Romance', 'genre_Science_Fiction',
       'genre_Thriller', 'genre_War', 'genre_Western',
       'spoken_languages_count', 'production_countries_count',
       'production_companies_count', 'Keywords_count', 'released'],
      dtype='object')

In [19]:
X_test_pred=model.predict_(X_test)
X_test_pred

array([ 3.9323590e+08,  9.4342278e+08,  7.3667725e+08,  7.8334125e+08,
        5.3463862e+08,  7.6994624e+08,  6.3474010e+08,  9.3637875e+08,
        5.5021107e+08,  2.0927644e+09, -8.2002300e+06], dtype=float32)

##  3.4.1 Predictions on new Raw Data

### 3.4.1.1 Load New Raw data

In [20]:
XX=raw_df.drop("revenue",axis=1)

In [21]:
XX.columns

Index(['Unnamed: 0', 'id', 'title', 'vote_average', 'vote_count', 'status',
       'release_date', 'runtime', 'adult', 'backdrop_path', 'budget',
       'homepage', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'Keywords'],
      dtype='object')

### 3.4.1.2 Transform XX in a gut input of predictable data

In [22]:
##### 3.4.1.2_1 Get Ct_transormer of the model
ct_data_transformer=model.ct_data_transformer

In [24]:
XX_transformed=Preprocess_predicted_data.preprocessing_pipeline(XX,ct_data_transformer)

In [25]:
XX_pred=model.predict_(XX_transformed)
XX_pred.shape,XX_transformed.shape

((50,), (50, 24))

# 3.5 Save Model

In [26]:
model.save_best_model("Best_best_model.pkl")

,Message
0,Model saved successfully.


# Bibliotheque Predict

# A sur le model déja entrainé depuis ce notebook

In [37]:
#definir une instance de la classe Predicto
Predictor=Predict__Model.Predicto(model)
#appliquer la methode predict de cette class à X_test
live_predictions = Predictor.predict(XX)
live_predictions.shape

(50,)

# B ) Sur un modéle à télécharger

In [39]:
#get trained model
loaded_model = joblib.load("Best_best_model.pkl")
#definir une instance de la classe Predicto
Predictor=Predict__Model.Predicto(model)
#appliquer la methode predict de cette class à X_test
predictions = Predictor.predict(XX)
predictions.shape

(50,)